98106434 -> Mohammad Khodadadi Aski
<br>
98105919 -> Reza Erfan Arani
<br>
98101566 -> Mohammadreza Daviran
<br>


We use the BeautifulSoup in order to extraxt data from the request we send to the website .
.strip method removes any unexpected and useless whitespace
In order to run the code , you should run the bellow cell to installl required libraries.

In [1]:
!pip install hazm
!pip install selenium
!pip install bs4

     ------------------------------------ 968.8/968.8 KB 358.8 kB/s eta 0:00:00
     ------------------------------------ 359.0/359.0 KB 360.3 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 365.0 kB/s eta 0:00:00
     -------------------------------------- 55.8/55.8 KB 970.1 kB/s eta 0:00:00
     -------------------------------------- 58.2/58.2 KB 339.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 128.2/128.2 KB 342.8 kB/s eta 0:00:00
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'


In [212]:
from __future__ import unicode_literals
from hazm import *

def run (input:str):
    normalizer = Normalizer()
    normilized_input=normalizer.normalize(input)
    tokenized_sent_input = sent_tokenize(input)
    tokenized_word_input = word_tokenize(input)
    stemmer = Stemmer()
    stemmed_input = stemmer.stem(normilized_input)
    # we did this , but we shouldnt use this data cause some data are lost during this pre-proccessing phase
    print(f'{tokenized_sent_input}\n{tokenized_word_input}')
    indices = [i for i, x in enumerate(tokenized_word_input) if x.startswith('دستور')]
    print(indices)
    lemmatizer = Lemmatizer()
    lemmed_input = lemmatizer.lemmatize(normilized_input)
    # we also have the same thing about lemmatizing (same as stemming)

In [228]:
# import time

In [229]:
def find_recipe(input_recipe, index):
    recipe = input_recipe[index:]
    newline_index = recipe.index(chr(10)) 
    recipe = recipe[newline_index + 1:]
    return recipe, [index + newline_index + 1, len(input_recipe)]

In [230]:
def findSpanIngredient(ingut: str, ingredients):
    span = []
    last = 0
    for ing in ingredients:
        try: 
            first = ingut.index(ing)
        except:
            first = last + 1
        while (last <= first):
            last = ingut.index(chr(10),last+1)
        span.append([first,last])
    return span

In [231]:
def find_ingredients(ingut : str):
    start_time = time.time()
    
    ingredients = []
    quantity = []
    span_ingredients = []
    if "مواد" in ingut.split("\n")[0]:
        starting_index = ingut.index("مواد")
    else:
        starting_index = 0
    prepare_spliter = ["طرز تهیه", "روش تهیه", "طرزتهیه"]
    ending_index = 0
    for element in prepare_spliter:
            try:
                ending_index = ingut.index(element)
                break
            except:
                continue
    
    ing_data = ingut[starting_index:ending_index]
    if "مواد" in ingut.split("\n")[0]:
        ing_data= ing_data.split("\n",1)[1]
        
    spliter = ['0', '۰','1', '۱', '2', '۲', '3', '۳', '4', '۴', '5', '۵', '6',
               '۶', '7', '۷', '8', '۸', '9', '۹', 'یک ', 'یک‌',
               'دو ', 'دو‌', 'سه ', 'سه‌','نیم', 'نیم‌', 'به مقدار لازم', 'به میزان لازم',
               'نصف ', 'نصف‌','مقداری', 'به میزان کافی', 'کمتر از', 'دوقاشق']
    
    while (True):    
        spliterIndex = []
        for i in spliter:
            try:
                spliterIndex.append(ing_data.index(i))
            except:
                spliterIndex.append(9223372036854775807)
        endline = 0
        try:
            endline = ing_data.index(chr(10)) 
        except:
            pass
        middle = min(spliterIndex)
        
        if (middle > endline):
            middle = endline
        
        ing = ing_data[0:middle].strip()
        quant = ing_data[middle:endline].strip()
        
        ing = ing.replace('=', '').strip()
        ing = ing.replace(':', '').strip()
        
        ingredients.append(ing)
        quantity.append(quant)
        ing_data = ing_data[endline+1:]
        if (len(ing_data) == 0):
            break
            
    recipe, span_recipe = find_recipe(ingut ,ending_index)
    span_ingredients = findSpanIngredient(ingut, ingredients)
    
    end_time = time.time()
    
    return (ingredients, quantity, recipe, span_ingredients, span_recipe, end_time - start_time)
   

In [232]:
def findRecipeIndex(s: str, ch: str):
    indexes = []
    first = 0
    while (True):
        try:
            first = s.index(ch, first)
            indexes.append(first)
            first += 1
        except:
            break
    return indexes

In [237]:
import time

lines = ''
with open('input.txt', 'r',  encoding='utf-8') as f:
    lines = f.read()
all_dicts = []
indexes = findRecipeIndex(lines, 'دستور شمار')
if len(indexes) == 0:
    indexes.append(0)
indexes.append(len(lines))
for i in range(len(indexes)-1):
    first = indexes[i]
    if indexes[i] != 0:
        first = lines.find(chr(10),first) + 1
    last = indexes[i+1]
    ingredients, quantity, recipe, span_ingredients, span_recipe, time_passed = find_ingredients(lines[first:last])
    final_dict = {}
    final_dict.update({'Ingredients': ingredients})
    final_dict.update({'quantity': quantity})
    final_dict.update({'recipie': recipe})
    final_dict.update({'span_ingredients': span_ingredients})
    final_dict.update({'span_recipie': span_recipe})
    final_dict.update({'time': time_passed})
    all_dicts.append(final_dict)
all_dicts

ValueError: substring not found